In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Parameters
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

plants = ['P1', 'P2']
distribution_centers = ['DC1', 'DC2', 'DC3']
plant_names = {'P1': 'Plant A', 'P2': 'Plant B'}
dc_names = {'DC1': 'Distribution Center 1', 'DC2': 'Distribution Center 2', 'DC3': 'Distribution Center 3'}

# Initialize lists to store data
data = []

# Fixed plant capacities
fixed_capacity = {
    'P1': 2000,  # Example fixed capacity for Plant A+ added optimization quantity
    'P2': 2500  # Example fixed capacity for Plant B+ added optimization quantity
}


# Initialize previous remaining inventory for each plant
previous_remaining_inventory = {
    'P1': 0,
    'P2': 0
}

# Generate data
for current_date in date_range:
    for plant_id in plants:
        # Use fixed plant capacity
        plant_capacity = fixed_capacity[plant_id]
        
        # Calculate the remaining capacity for the current day
        remaining_capacity = previous_remaining_inventory[plant_id] + plant_capacity
        
        for dc_id in distribution_centers:
            order_id = len(data) + 1
            
            # Add uncertainty to demand
            base_demand = np.random.randint(500, 1500)
            demand_fluctuation = np.random.normal(0, 50)  # Normal distribution with mean 0 and std 50
            demand = int(max(0, base_demand + demand_fluctuation))  # Ensure non-negative demand
            
            # Ensure supply does not exceed remaining capacity and adjust demand to available capacity
            base_supply = np.random.randint(1000, 2000)
            supply_fluctuation = np.random.normal(0, 100)  # Normal distribution with mean 0 and std 100
            supply = int(max(0, base_supply + supply_fluctuation))  # Ensure non-negative supply
            supply = min(supply, remaining_capacity, demand)
            
            # Add uncertainty to unit cost
            unit_cost = np.random.uniform(4.00, 6.00)
            cost_fluctuation = np.random.uniform(-0.50, 0.50)  # Random fluctuation between -0.50 and 0.50
            unit_cost = round(unit_cost + cost_fluctuation, 2)  # Ensure two decimal places
            
            # Initialize quantity shipped and fulfillment status
            quantity_shipped = 0
            
            # Ensure that plant uses remaining capacity if possible
            if remaining_capacity >= demand:
                quantity_shipped = demand
                remaining_capacity -= quantity_shipped
                fulfillment_status = 'Fulfilled'
            else:
                quantity_shipped = remaining_capacity
                remaining_capacity = 0
                if quantity_shipped > 0:
                    fulfillment_status = 'Partially Fulfilled'
            
            # Ensure remaining capacity does not go negative
            remaining_capacity = max(remaining_capacity, 0)
            
            shipping_cost = quantity_shipped * unit_cost
            
            data.append({
                'Order_ID': order_id,
                'Date': current_date,
                'Plant_ID': plant_id,
                'Plant_Name': plant_names[plant_id],
                'Plant_Capacity': plant_capacity,
                'Remaining Inventory level': remaining_capacity,
                'Distribution_Center_ID': dc_id,
                'Distribution_Center_Name': dc_names[dc_id],
                'Demand': demand,
                'Supply': supply,
                'Unit_Cost': unit_cost,
                'Quantity_Shipped': quantity_shipped,
                'Shipping_Cost': shipping_cost,
            })
            
            # If there's remaining capacity, continue fulfilling other demands
            if remaining_capacity <= 0:
                break
        
        # Update previous remaining inventory for the next day's use
        previous_remaining_inventory[plant_id] = remaining_capacity

# Convert to DataFrame
df = pd.DataFrame(data)

df['Status_final'] = np.where(df['Demand'] - df['Supply'] > 0, 'partially_Fulfilled', 'Fulfilled')
df['Demand_surplus'] = df['Demand'] - df['Quantity_Shipped']

# Save the DataFrame to a CSV file
df.to_csv('Production_distribution_initial_dataset.csv', index=False)

print("Initial dataset")

Initial dataset


In [3]:
df

,Order_ID,Date,Plant_ID,Plant_Name,Plant_Capacity,Remaining Inventory level,Distribution_Center_ID,Distribution_Center_Name,Demand,Supply,Unit_Cost,Quantity_Shipped,Shipping_Cost,Status_final,Demand_surplus
0,1,2021-01-01,P1,Plant A,2000,1230,DC1,Distribution Center 1,770,770,4.78,770,3680.60,Fulfilled,0
1,2,2021-01-01,P1,Plant A,2000,0,DC2,Distribution Center 2,1366,1230,5.24,1230,6445.20,partially_Fulfilled,136
2,3,2021-01-01,P2,Plant B,2500,1434,DC1,Distribution Center 1,1066,1066,5.67,1066,6044.22,Fulfilled,0
3,4,2021-01-01,P2,Plant B,2500,820,DC2,Distribution Center 2,614,614,5.59,614,3432.26,Fulfilled,0
4,5,2021-01-01,P2,Plant B,2500,0,DC3,Distribution Center 3,1401,820,4.42,820,3624.40,partially_Fulfilled,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7883,7884,2024-12-31,P1,Plant A,2000,485,DC2,Distribution Center 2,893,893,5.01,893,4473.93,Fulfilled,0
7884,7885,2024-12-31,P1,Plant A,2000,0,DC3,Distribution Center 3,921,485,5.14,485,2492.90,partially_Fulfilled,436
7885,7886,2024-12-31,P2,Plant B,2500,1302,DC1,Distribution Center 1,1198,1198,5.30,1198,6349.40,Fulfilled,0
7886,7887,2024-12-31,P2,Plant B,2500,431,DC2,Distribution Center 2,871,871,4.56,871,3971.76,Fulfilled,0


In [4]:
df["Demand"].sum()

7871073

In [5]:
df["Demand_surplus"].sum()

1296573

In [6]:
#the demands of each distribution center are fulfilled by the end of the day

# Aggregate daily demand and quantity shipped per distribution center
daily_aggregates = df.groupby(['Date', 'Distribution_Center_ID']).agg(
    total_demand=('Demand', 'sum'),
    total_quantity_shipped=('Quantity_Shipped', 'sum')
).reset_index()

# Determine if the demand is fulfilled
daily_aggregates['Demand_Fulfilled'] = daily_aggregates['total_quantity_shipped'] >= daily_aggregates['total_demand']


# Optionally, count the number of days demand was fully met per distribution center
demand_fulfillment_summary = daily_aggregates.groupby('Distribution_Center_ID')['Demand_Fulfilled'].value_counts().unstack().fillna(0)

demand_fulfillment_summary


Demand_Fulfilled,False,True
Distribution_Center_ID,,
DC1,0.0,1461.0
DC2,804.0,657.0
DC3,1230.0,158.0


In [7]:
df["Quantity_Shipped"].sum()

6574500

In [8]:
df['Status_final'].value_counts()

Fulfilled              4739
partially_Fulfilled    3149
Name: Status_final, dtype: int64

In [9]:
df.groupby(['Plant_ID'])['Unit_Cost'].describe()

,count,mean,std,min,25%,50%,75%,max
Plant_ID,,,,,,,,
P1,3658.0,5.008054,0.633672,3.54,4.52,5.00,5.5000,6.47
P2,4230.0,4.993480,0.646191,3.52,4.50,4.98,5.4875,6.45


In [10]:
df.groupby(['Distribution_Center_Name'])['Demand'].describe()

,count,mean,std,min,25%,50%,75%,max
Distribution_Center_Name,,,,,,,,
Distribution Center 1,2922.0,1000.405202,293.650035,412.0,749.0,998.5,1254.00,1637.0
Distribution Center 2,2922.0,992.659822,297.164631,397.0,733.0,985.5,1245.75,1599.0
Distribution Center 3,2044.0,1001.632583,297.304038,391.0,753.0,992.0,1261.00,1627.0


In [11]:
#Plant capacity utilization
df.groupby("Date").agg({
    'Quantity_Shipped':'sum'
})
#total day==1461
# we have Fixed plant capacity
total_plant_production_in_1461_days=(2000+2500)*1461
plant_capacity_utilization=(df["Quantity_Shipped"].sum()/total_plant_production_in_1461_days)*100

In [12]:
# 1. Total Quantity Shipped
print("Optimized result")
total_quantity_shipped = df['Quantity_Shipped'].sum()

# total demand 
total_demand = df["Demand"].sum()

# 2. Total Shipping Cost
total_shipping_cost = df['Shipping_Cost'].sum()

# 3. Average Fulfillment Rate
fulfillment_rate = df['Status_final'].value_counts(normalize=True).get('Fulfilled', 0) * 100

# 4. Average Unit Cost
average_unit_cost = df['Unit_Cost'].mean()

# 5. Total Demand vs. Supply
total_demand = df['Demand'].sum()
total_supply = df['Supply'].sum()
demand_vs_supply_ratio = total_demand / total_supply

# 6. Capacity Utilization
capacity_utilization=plant_capacity_utilization


# 7. Average Shipping Cost per Unit
average_shipping_cost_per_unit = df['Shipping_Cost'].sum() / df['Quantity_Shipped'].sum()

# 8. Percentage of Orders Fulfilled Fully
percentage_fulfilled_fully = df[df['Status_final'] == 'Fulfilled'].shape[0] / df.shape[0] * 100

# 9. Average Remaining Capacity per Day

# 10. Demand Fulfillment Efficiency
demand_fulfillment_efficiency = (df['Quantity_Shipped'].sum() / df['Demand'].sum()) * 100

print("Total Quantity Shipped:", total_quantity_shipped)
print("Total Demand :",total_demand)
print("Total Shipping Cost:", total_shipping_cost)
print("Average Fulfillment Rate (%):", fulfillment_rate)
print("Average Unit Cost:", average_unit_cost)
print("Demand vs. Supply Ratio:", demand_vs_supply_ratio)
print("Capacity Utilization (%):", capacity_utilization)
print("Average Shipping Cost per Unit:", average_shipping_cost_per_unit)
print("Percentage of Orders Fulfilled Fully (%):", percentage_fulfilled_fully)
print("Demand Fulfillment Efficiency (%):", demand_fulfillment_efficiency)


Optimized result
Total Quantity Shipped: 6574500
Total Demand : 7871073
Total Shipping Cost: 32886270.68
Average Fulfillment Rate (%): 60.078600405679516
Average Unit Cost: 5.000238336713997
Demand vs. Supply Ratio: 1.2186504272281542
Capacity Utilization (%): 100.0
Average Shipping Cost per Unit: 5.0020945592820745
Percentage of Orders Fulfilled Fully (%): 60.078600405679516
Demand Fulfillment Efficiency (%): 83.52736659919175


## Linear programming for both plant demand fulfillment optimization

In [13]:
import pulp

# Define the problem
problem = pulp.LpProblem("Demand_fulfillment_Optimization", pulp.LpMinimize)

# Decision variables: x1 and x2 represent the number of units produced by Plant A and Plant B, respectively
x1 = pulp.LpVariable('x1', lowBound=0, cat='Integer')
x2 = pulp.LpVariable('x2', lowBound=0, cat='Integer')

# Example unit production costs for Plant A and Plant B
c1 = 5  # Unit production cost for Plant A
c2 = 5  # Unit production cost for Plant B

# Capacity expansion costs per unit
capacity_cost_a = 10
capacity_cost_b = 12

# Total cost for each unit produced considering both production and capacity cost
total_cost_a = c1 + capacity_cost_a
total_cost_b = c2 + capacity_cost_b

# Objective function: Minimize total cost
problem += total_cost_a * x1 + total_cost_b * x2, "Total_Cost"

# Example demands for the 3 distribution centers
D1 = 992.0  # Demand for Distribution Center 1
D2 = 1003.5  # Demand for Distribution Center 2
D3 = 1001.5 # Demand for Distribution Center 3

# Constraints: Ensure total production meets or exceeds the demand for each distribution center
problem += x1 + x2 >= 0.8*(D1+D2+D3),"Demand Constraint"
problem+=x1>=0.54*(x1+x2),'60 percent contribution of x1'
problem+=x2>=0.46*(x1+x2),'40 percent contribution of x2'
#problem += x1 + x2 >= D1+D2+D3, "Demand_Constraint_1"

# Solve the problem
problem.solve()

# Output the results
print(f"Status: {pulp.LpStatus[problem.status]}")
print(f"Optimal production for Plant A: {x1.varValue} units")
print(f"Optimal production for Plant B: {x2.varValue} units")
print(f"Minimum total cost: ${pulp.value(problem.objective)}")


Status: Optimal
Optimal production for Plant A: 1296.0 units
Optimal production for Plant B: 1104.0 units
Minimum total cost: $38208.0


In [14]:
# Parameters
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

plants = ['P1', 'P2']
distribution_centers = ['DC1', 'DC2', 'DC3']
plant_names = {'P1': 'Plant A', 'P2': 'Plant B'}
dc_names = {'DC1': 'Distribution Center 1', 'DC2': 'Distribution Center 2', 'DC3': 'Distribution Center 3'}

# Initialize lists to store data
data = []

# Fixed plant capacities
fixed_capacity = {
    'P1': 2000+x1.varValue,  # Example fixed capacity for Plant A+ added optimization quantity
    'P2': 2500+x2.varValue  # Example fixed capacity for Plant B+ added optimization quantity
}


# Initialize previous remaining inventory for each plant
previous_remaining_inventory = {
    'P1': 0,
    'P2': 0
}

# Generate data
for current_date in date_range:
    for plant_id in plants:
        # Use fixed plant capacity
        plant_capacity = fixed_capacity[plant_id]
        
        # Calculate the remaining capacity for the current day
        remaining_capacity = previous_remaining_inventory[plant_id] + plant_capacity
        
        for dc_id in distribution_centers:
            order_id = len(data) + 1
            
            # Add uncertainty to demand
            base_demand = np.random.randint(500, 1500)
            demand_fluctuation = np.random.normal(0, 50)  # Normal distribution with mean 0 and std 50
            demand = int(max(0, base_demand + demand_fluctuation))  # Ensure non-negative demand
            
            # Ensure supply does not exceed remaining capacity and adjust demand to available capacity
            base_supply = np.random.randint(1000, 2000)
            supply_fluctuation = np.random.normal(0, 100)  # Normal distribution with mean 0 and std 100
            supply = int(max(0, base_supply + supply_fluctuation))  # Ensure non-negative supply
            supply = min(supply, remaining_capacity, demand)
            
            # Add uncertainty to unit cost
            unit_cost = np.random.uniform(4.00, 6.00)
            cost_fluctuation = np.random.uniform(-0.50, 0.50)  # Random fluctuation between -0.50 and 0.50
            unit_cost = round(unit_cost + cost_fluctuation, 2)  # Ensure two decimal places
            
            # Initialize quantity shipped and fulfillment status
            quantity_shipped = 0
            
             # Ensure that plant uses remaining capacity if possible
            if remaining_capacity >= demand:
                quantity_shipped = demand
                remaining_capacity -= quantity_shipped
                fulfillment_status = 'Fulfilled'
            else:
                quantity_shipped = remaining_capacity
                remaining_capacity = 0
                if quantity_shipped > 0:
                    fulfillment_status = 'Partially Fulfilled'
            
            # Ensure remaining capacity does not go negative
            remaining_capacity = max(remaining_capacity, 0)
            
            shipping_cost = quantity_shipped * unit_cost
            
            data.append({
                'Order_ID': order_id,
                'Date': current_date,
                'Plant_ID': plant_id,
                'Plant_Name': plant_names[plant_id],
                'Plant_Capacity': plant_capacity,
                'Remaining Inventory level': remaining_capacity,
                'Distribution_Center_ID': dc_id,
                'Distribution_Center_Name': dc_names[dc_id],
                'Demand': demand,
                'Supply': supply,
                'Unit_Cost': unit_cost,
                'Quantity_Shipped': quantity_shipped,
                'Shipping_Cost': shipping_cost,
            })
            
            # If there's remaining capacity, continue fulfilling other demands
            if remaining_capacity <= 0:
                break
        
        # Update previous remaining inventory for the next day's use
        previous_remaining_inventory[plant_id] = remaining_capacity

# Convert to DataFrame
df1 = pd.DataFrame(data)

df1['Status_final'] = np.where(df1['Demand'] - df1['Supply'] > 0, 'partially_Fulfilled', 'Fulfilled')
df1['Demand_surplus'] = df1['Demand'] - df1['Quantity_Shipped']

# Save the DataFrame to a CSV file
df1.to_csv('Production_distribution_optimized_dataset.csv', index=False)

print("Optimized_data")

Optimized_data


In [15]:
df1.head(20)

,Order_ID,Date,Plant_ID,Plant_Name,Plant_Capacity,Remaining Inventory level,Distribution_Center_ID,Distribution_Center_Name,Demand,Supply,Unit_Cost,Quantity_Shipped,Shipping_Cost,Status_final,Demand_surplus
0,1,2021-01-01,P1,Plant A,3296.0,2321.0,DC1,Distribution Center 1,975,975.0,4.36,975.0,4251.00,Fulfilled,0.0
1,2,2021-01-01,P1,Plant A,3296.0,1370.0,DC2,Distribution Center 2,951,951.0,4.29,951.0,4079.79,Fulfilled,0.0
2,3,2021-01-01,P1,Plant A,3296.0,0.0,DC3,Distribution Center 3,1398,1120.0,5.82,1370.0,7973.40,partially_Fulfilled,28.0
3,4,2021-01-01,P2,Plant B,3604.0,2742.0,DC1,Distribution Center 1,862,862.0,4.98,862.0,4292.76,Fulfilled,0.0
4,5,2021-01-01,P2,Plant B,3604.0,2104.0,DC2,Distribution Center 2,638,638.0,4.48,638.0,2858.24,Fulfilled,0.0
5,6,2021-01-01,P2,Plant B,3604.0,881.0,DC3,Distribution Center 3,1223,1223.0,4.62,1223.0,5650.26,Fulfilled,0.0
6,7,2021-01-02,P1,Plant A,3296.0,1924.0,DC1,Distribution Center 1,1372,1372.0,5.97,1372.0,8190.84,Fulfilled,0.0
7,8,2021-01-02,P1,Plant A,3296.0,1156.0,DC2,Distribution Center 2,768,768.0,5.11,768.0,3924.48,Fulfilled,0.0
8,9,2021-01-02,P1,Plant A,3296.0,0.0,DC3,Distribution Center 3,1168,1156.0,5.46,1156.0,6311.76,partially_Fulfilled,12.0
9,10,2021-01-02,P2,Plant B,3604.0,3732.0,DC1,Distribution Center 1,753,753.0,5.10,753.0,3840.30,Fulfilled,0.0


In [16]:
df1[["Date",'Plant_ID',"Plant_Capacity","Remaining Inventory level","Distribution_Center_ID","Status_final","Demand","Quantity_Shipped","Demand_surplus"]].head(30)

,Date,Plant_ID,Plant_Capacity,Remaining Inventory level,Distribution_Center_ID,Status_final,Demand,Quantity_Shipped,Demand_surplus
0,2021-01-01,P1,3296.0,2321.0,DC1,Fulfilled,975,975.0,0.0
1,2021-01-01,P1,3296.0,1370.0,DC2,Fulfilled,951,951.0,0.0
2,2021-01-01,P1,3296.0,0.0,DC3,partially_Fulfilled,1398,1370.0,28.0
3,2021-01-01,P2,3604.0,2742.0,DC1,Fulfilled,862,862.0,0.0
4,2021-01-01,P2,3604.0,2104.0,DC2,Fulfilled,638,638.0,0.0
5,2021-01-01,P2,3604.0,881.0,DC3,Fulfilled,1223,1223.0,0.0
6,2021-01-02,P1,3296.0,1924.0,DC1,Fulfilled,1372,1372.0,0.0
7,2021-01-02,P1,3296.0,1156.0,DC2,Fulfilled,768,768.0,0.0
8,2021-01-02,P1,3296.0,0.0,DC3,partially_Fulfilled,1168,1156.0,12.0
9,2021-01-02,P2,3604.0,3732.0,DC1,Fulfilled,753,753.0,0.0


In [17]:
#the demands of each distribution center are fulfilled by the end of the day

# Aggregate daily demand and quantity shipped per distribution center
daily_aggregates = df1.groupby(['Date', 'Distribution_Center_ID']).agg(
    total_demand=('Demand', 'sum'),
    total_quantity_shipped=('Quantity_Shipped', 'sum')
).reset_index()

# Determine if the demand is fulfilled
daily_aggregates['Demand_Fulfilled'] = daily_aggregates['total_quantity_shipped'] >= daily_aggregates['total_demand']


# Optionally, count the number of days demand was fully met per distribution center
demand_fulfillment_summary = daily_aggregates.groupby('Distribution_Center_ID')['Demand_Fulfilled'].value_counts().unstack().fillna(0)

demand_fulfillment_summary


Demand_Fulfilled,False,True
Distribution_Center_ID,,
DC1,0.0,1461.0
DC2,0.0,1461.0
DC3,2.0,1459.0


In [18]:
df1["Status_final"].value_counts()

Fulfilled              7613
partially_Fulfilled    1153
Name: Status_final, dtype: int64

In [19]:
#Plant capacity utilization
df1.groupby("Date").agg({
    'Quantity_Shipped':'sum'
})
#total day==1461
# we have Fixed plant capacity
total_plant_production_in_1461_days=(2000+x1.varValue+2500+x2.varValue )*1461
plant_capacity_utilization=(df1["Quantity_Shipped"].sum()/total_plant_production_in_1461_days)*100

In [20]:
# 1. Total Quantity Shipped
print("Optimized result")
total_quantity_shipped = df1['Quantity_Shipped'].sum()

# total demand 
total_demand = df1["Demand"].sum()

# 2. Total Shipping Cost
total_shipping_cost = df1['Shipping_Cost'].sum()

# 3. Average Fulfillment Rate
fulfillment_rate = df1['Status_final'].value_counts(normalize=True).get('Fulfilled', 0) * 100

# 4. Average Unit Cost
average_unit_cost = df1['Unit_Cost'].mean()

# 5. Total Demand vs. Supply
total_demand = df1['Demand'].sum()
total_supply = df1['Supply'].sum()
demand_vs_supply_ratio = total_demand / total_supply

# 6. Capacity Utilization
capacity_utilization=plant_capacity_utilization


# 7. Average Shipping Cost per Unit
average_shipping_cost_per_unit = df1['Shipping_Cost'].sum() / df1['Quantity_Shipped'].sum()

# 8. Percentage of Orders Fulfilled Fully
percentage_fulfilled_fully = df1[df1['Status_final'] == 'Fulfilled'].shape[0] / df1.shape[0] * 100

# 9. Average Remaining Capacity per Day

# 10. Demand Fulfillment Efficiency
demand_fulfillment_efficiency = (df1['Quantity_Shipped'].sum() / df1['Demand'].sum()) * 100

print("Total Quantity Shipped:", total_quantity_shipped)
print("Total Demand :",total_demand)
print("Total Shipping Cost:", total_shipping_cost)
print("Average Fulfillment Rate (%):", fulfillment_rate)
print("Average Unit Cost:", average_unit_cost)
print("Demand vs. Supply Ratio:", demand_vs_supply_ratio)
print("Capacity Utilization (%):", capacity_utilization)
print("Average Shipping Cost per Unit:", average_shipping_cost_per_unit)
print("Percentage of Orders Fulfilled Fully (%):", percentage_fulfilled_fully)
print("Demand Fulfillment Efficiency (%):", demand_fulfillment_efficiency)


Optimized result
Total Quantity Shipped: 8786378.0
Total Demand : 8786418
Total Shipping Cost: 43865846.04
Average Fulfillment Rate (%): 86.84690851015287
Average Unit Cost: 4.993133698380105
Demand vs. Supply Ratio: 1.0253015487081418
Capacity Utilization (%): 87.15866638891369
Average Shipping Cost per Unit: 4.992483369142552
Percentage of Orders Fulfilled Fully (%): 86.84690851015287
Demand Fulfillment Efficiency (%): 99.99954475191142
